### 人工市場による時系列生成〜適応度の算出

#### 人工市場上のメソッド

In [100]:
import matplotlib.pyplot as plt
import networkx as nx
import random as rnd
import numpy as np
import collections
%matplotlib inline

N = 5
p_sell =0.4
p_buy =0.4
num_play = 365

class Agent:
    def  __init__(self,id):
        self.id = id
        self.strategy = None # hold
        self.buy = p_buy
        self.sell = p_sell
def  decision(agent_list):
    for focal in agent_list:
        r = rnd.random()
        if 0<= r<focal.buy:
            focal.strategy = 1
        elif focal.buy<=r<=focal.buy+focal.sell:
            focal.strategy = -1
        else: 
            focal.strategy= 0

def count(agent_list):
    delta = sum(list(map(lambda agent:agent.strategy, agent_list)))
    return delta

def find_hub(G):
    degree_sequence = sorted([d for n, d in G.degree().items()])
    degreeCount = collections.Counter(degree_sequence)
    #calculate path lenght to the nearest influencer and inflencer node. 
    par = 0.7 # upper 100*(1-par)%  
    hub = [n for n, d in G.degree().items() if d>max(degreeCount.keys())*par]
    return hub

def find_influencer(G, hub):
    inf = []
    length = []
    store = [] # [influencer node, path length to influencer]
    isolates = nx.isolates(G)
    l1 = []
    l2 = []
    # find nodes which have no path to hub, then store in sub  
    sub=[] 
    for n in range(N):
        ns = nx.node_connected_component(G,n)
        if not ns & set(hub):sub.append(n)    

    tmp = []
    for n in G.degree().keys():
        if n in hub or n in isolates or n in sub:
            store.append([n,0])
        else:
            for h in hub:
                try:
                    tmp.append(nx.shortest_path_length(G,n,h))
                except:
                    pass
            mi = min(tmp)
            store.append([hub[tmp.index(mi)], mi])
            tmp.clear()
    num = [i for i in range(N)]
    #di = dict(zip(num,store)) # di = {node num:[nearest influencer, path length to nearest influencer]}
    influencers = [s[0] for s in store]
    lengths = [s[1] for s in store]
    return influencers, lengths

def update_strategy(agent_list, influencers, lengths):
    for focal, i,l in zip(agent_list, influencers,lengths):
        r = rnd.random()
        p = 1-0.1*l
        if 0<=r<p:
            focal.strategy  =  agent_list[i].strategy

def Graph_to_gene(G):
    mat   = np.array(nx.to_numpy_matrix(G))
    print(mat)
    gene = []
    for i in range(N-1):
        tmp = mat[i,i+1:]
        gene.extend(tmp)
    gene = list(map(int, gene))
    return gene

def gene_to_graph(gene):
    mat = np.empty((N,N))
    loc = 0
    gene = np.array([i for i in range(10)][::-1])
    for i in range(0,N):
        mat[i,i+1:]  = gene[loc:loc+(N-i-1)]
        loc +=N-i-1
    mat  = mat+ mat.T
    G = nx.from_numpy_matrix(mat)
    return G
    

#### 人工市場上の時系列生成

In [112]:
def priceSeries(G): 
    agent_list = [Agent(id) for id in range(N)]
    price = 0
    prices = [price]
    hub = find_hub(G)
    inflencers, lengths = find_influencer(G, hub)
    for play in range(num_play):
        decision(agent_list)
        update_strategy(agent_list, inflencers, lengths)
        delta = count(agent_list)
        price +=delta
        prices.append(price)
    
    #draw price movement
#     plt.plot(prices, label="price")
#     plt.title("Simulation Result")
#     plt.ylabel("price")
#     plt.xlabel("time step")
#     plt.grid("on")
#     plt.legend()
#     plt.show()
#     print (transform_into_gene(G))
    return prices

In [113]:
import datetime 
import pandas as pd 
import pandas_datareader.data as web
import scipy.stats as sp

def evaluate(individual):
    # statistics of SP500
    kuto = 7.653
    sig1 = 0.79
    sig3 = 0.056
    
    G = gene_to_graph(individual)
    prices = priceSeries(G)
    
    # kutosis
    kutosis = sp.kurtosis(prices)
    # sigma
    df  =pd.Series(prices)
    x  = df.pct_change()
    in_1sigma = len([i for i in x if np.average(x)-np.std(x)<i<np.average(x)+np.std(x)])/len(x)
    out_3sigma = len([i for i in x if i<np.average(x)-np.std(x)*2 or i>np.average(x)+np.std(x)*2])/len(x)
    a = abs(kuto -kutosis)
    b = abs(sig1-in_1sigma)
    c = abs(sig3-out_3sigma)
    
    return a,b,c

In [114]:
import array
import random
import json

import numpy

from math import sqrt

from deap import algorithms
from deap import base
from deap import benchmarks
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(-1.0,-1.0,-1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox  = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, int(1/2*N*(N-1)))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("select", tools.selNSGA2)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("evaluate",evaluate,)


def main(seed=None):
    random.seed(seed)

    NGEN = 10
    MU = 100
    CXPB = 0.9

    stats = tools.Statistics(lambda ind: ind.fitness.values) #  argument of stats is poplation. 
    stats.register("avg", numpy.mean, axis=0)
#     stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)
    
    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "min", "avg", "max"
    
    pop = toolbox.population(n=MU)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # This is just to assign the crowding distance to the individuals
    # no actual selection is done
    pop = toolbox.select(pop, len(pop))
    
    record = stats.compile(pop)
    logbook.record(gen=0, evals=len(invalid_ind), **record)
    print(logbook.stream)

    # Begin the generational process
    for gen in range(1, NGEN):
        # Vary the population
        offspring = tools.selTournamentDCD(pop, len(pop))
        offspring = [toolbox.clone(ind) for ind in offspring]
        
        for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
            if random.random() <= CXPB:
                toolbox.mate(ind1, ind2)
            
            toolbox.mutate(ind1)
            toolbox.mutate(ind2)
            del ind1.fitness.values, ind2.fitness.values
        
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Select the next generation population
        pop = toolbox.select(pop + offspring, MU)
        record = stats.compile(pop)
        logbook.record(gen=gen, evals=len(invalid_ind), **record)
        print(logbook.stream)

    #print("Final population hypervolume is %f" % hypervolume(pop, [11.0, 11.0]))

    return pop, logbook


/Users/puca/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/Users/puca/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [115]:
if __name__ == "__main__":
    main()

gen	evals	min                                  	avg                                  	max                                  
0  	100  	[ 6.92804857  0.79        0.056     ]	[ 8.37093494  0.79        0.056     ]	[ 9.23712223  0.79        0.056     ]
1  	100  	[ 6.12476742  0.79        0.056     ]	[ 7.90445191  0.79        0.056     ]	[ 8.41599553  0.79        0.056     ]


KeyboardInterrupt: 